In [14]:
from langchain_ollama import ChatOllama

# Initialize the ChatOllama object
llm = ChatOllama(
    model = "llama3.1",
    temperature = 0.8,
    num_predict = 256,
)


In [24]:
llm.invoke("tell me a joke about woodpeckers")

AIMessage(content="Here's one:\n\nWhy did the woodpecker go to the doctor?\n\nBecause it had a screw loose!\n\nGet it?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-09-19T03:33:46.9678629Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6433309500, 'load_duration': 29608500, 'prompt_eval_count': 19, 'prompt_eval_duration': 1968544000, 'eval_count': 26, 'eval_duration': 4435548000}, id='run-2145a36d-a967-43c7-979f-73a339370281-0', usage_metadata={'input_tokens': 19, 'output_tokens': 26, 'total_tokens': 45})

In [17]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='A cat', punchline='Why did the cat join a band? Because it wanted to be the purr-cussionist!', rating=8)

In [26]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

{'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!',
 'rating': 8,
 'setup': 'I just heard that cat can play music'}

In [35]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

{'punchline': 'Why did the cat join a band?', 'rating': 8, 'setup': 'Because it wanted to be the purr-cussionist!'}


In [36]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'punchline': 'Because they have a pecking interest in trees!',
 'rating': '8',
 'setup': 'Why do woodpeckers like trees?'}

In [37]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'punchline': 'Because it has a snappy jaw!',
 'rating': 8,
 'setup': 'Why did the crocodile go to the party?'}

In [39]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-09-19T03:56:39.3302123Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'Joke', 'arguments': {'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!', 'rating': 8, 'setup': 'Why did the cat join a band?'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 13178793500, 'load_duration': 2160804200, 'prompt_eval_count': 206, 'prompt_eval_duration': 523082000, 'eval_count': 56, 'eval_duration': 10493172000}, id='run-9fdc170a-b2a7-4aa7-a434-8b91816ab0e9-0', tool_calls=[{'name': 'Joke', 'args': {'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!', 'rating': 8, 'setup': 'Why did the cat join a band?'}, 'id': '5fce4045-0555-4a94-b14d-61ffeabb6636', 'type': 'tool_call'}], usage_metadata={'input_tokens': 206, 'output_tokens': 56, 'total_tokens': 262}),
 'parsed': {'punc

In [81]:
from langchain_core.prompts import PromptTemplate
from langchain.pydantic_v1 import  Field, validator
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain.output_parsers import  PydanticOutputParser

# Initialize the ChatOllama object
llm = ChatOllama(
    model = "llama3.1",
    temperature = 0,
)

# Define your desired data structure with improved descriptions
class Joke(BaseModel):
    setup: str = Field(description="The initial part of the joke that sets the scene or asks a question. It should end with a question mark.")
    punchline: str = Field(description="The final part of the joke that delivers the humor or surprise. It should be concise and related to the setup.")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Setup must end with a question mark!")
        return field


# Set up the parser
parser = PydanticOutputParser(pydantic_object=Joke)

prompt_template = """You are a witty comedian tasked with creating a short, funny joke.

Instructions:
1. Create a joke with a setup (question) and a punchline (answer).
2. The setup should end with a question mark.
3. The punchline should be clever and related to the setup.
4. Keep both parts concise and clear.

Format your response according to these instructions:
{format_instructions}

Now, please generate a joke based on this query: {query}
"""

# Improved prompt template
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create the chain
chain = prompt | llm | parser

# Query for a joke
joke_query = "Tell me a joke about programming."
result = chain.invoke({"query": joke_query})
print(result)

c:\Users\Diana\OneDrive\Documents\ingridientsGPT\venv\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The initial part of the joke that sets the scene or asks a question. It should end with a question mark.' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\Diana\OneDrive\Documents\ingridientsGPT\venv\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='The final part of the joke that delivers the humor or surprise. It should be concise and related to the setup.' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


setup='Why do programmers prefer dark mode?' punchline='Because light attracts bugs.'


In [82]:
another_query = "Tell me a joke about cats."
result = chain.invoke({"query": another_query})
print(result)

another_query = "Tell me a joke about simba and leo 2 cats."
result = chain.invoke({"query": another_query})
print(result)

setup='Why did the cat join a band?' punchline='Because it wanted to be the purr-cussionist!'
setup='Why did Simba and Leo go to the vet?' punchline='Because they were having cat-astrophic health issues!'
